### Helpers

In [2]:
#Packages for NLP
import pickle
import numpy as np
from tqdm import tqdm
import pandas as pd
import json
import string
import nltk
from nltk.corpus import stopwords
from  nltk import FreqDist
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import spacy
from textblob import TextBlob
from textblob import Word
from wordcloud import WordCloud, ImageColorGenerator
import warnings
warnings.filterwarnings('ignore')

nlp = spacy.load("en_core_web_sm")

## Getting dataframes

In [8]:
#Importing big dataframes and dataframes per year
df = pd.read_pickle('../Data-Cleaning-EDA/cleaned_full_data.pkl')
year_df = pd.read_pickle('../Data-Cleaning-EDA/cleaned_year_data.pkl')

In [9]:
print(df.shape)
df.head()

(27514, 21)


,Artist,Title,Album,name,duration_ms,popularity,num_markets,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,Year_release,Cleaned_lyrics
0,DJ Jazzy Jeff & The Fresh Prince,Time to Chill,"He’s The DJ, I’m The Rapper",Time to Chill,266040,24,79,0.735,0.559,7,...,1,0.161,0.00172,0.000020,0.0648,0.887,170.385,4,1988,for years i have been waitin and hesitatin to ...
1,Machine Gun Kelly,Candy,Hotel Diablo,Candy (feat. Trippie Redd),156719,78,79,0.744,0.638,5,...,0,0.049,0.00829,0.000000,0.1820,0.480,98.019,4,2019,haha big 14 you know what the fuck going on ...
2,Big Pun,You Ain’t a Killer,Capital Punishment,You Ain't a Killer,254866,57,79,0.724,0.887,6,...,1,0.393,0.13400,0.000000,0.1120,0.745,90.945,4,1998,the harsh realities of life have taken toll ev...
3,Das EFX,Buck-Buck,Hold It Down,Buck-Buck,193626,22,78,0.791,0.783,6,...,1,0.416,0.29300,0.000000,0.2950,0.531,95.677,4,1995,skoob and dray giving shoutouts chorus well ...
4,Kool Moe Dee,To the Beat Y’all,Funke Funke Wisdom,To The Beat Y'All,201266,11,77,0.801,0.481,0,...,1,0.232,0.01930,0.000107,0.3950,0.578,108.597,4,1991,to the beat to the beat y'all to the beat y'a...
